In [1]:
from preprocess import load_datasets, preprocess_hepatitis_datasets

In [2]:
# Load hepatitis train and test dataframes

train_dfs = load_datasets('datasetsCBR/hepatitis/hepatitis.fold.*.train.arff')
test_dfs = load_datasets('datasetsCBR/hepatitis/hepatitis.fold.*.test.arff')

train_dfs[0].head()


,AGE,SEX,STEROID,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,LIVER_BIG,LIVER_FIRM,SPLEEN_PALPABLE,SPIDERS,ASCITES,VARICES,BILIRUBIN,ALK_PHOSPHATE,SGOT,ALBUMIN,PROTIME,HISTOLOGY,Class
0,38.0,b'female',b'no',b'no',b'yes',b'yes',b'yes',b'yes',b'yes',b'no',b'yes',b'yes',b'yes',1.2,118.0,16.0,2.8,NaN,b'yes',b'DIE'
1,54.0,b'female',b'no',b'no',b'yes',b'yes',b'no',b'yes',b'no',b'no',b'no',b'yes',b'no',1.2,85.0,92.0,3.1,66.0,b'yes',b'LIVE'
2,58.0,b'male',b'yes',b'no',b'yes',b'no',b'no',b'yes',b'yes',b'no',b'yes',b'no',b'no',1.4,175.0,55.0,2.7,36.0,b'no',b'LIVE'
3,23.0,b'female',b'yes',b'no',b'no',b'no',b'no',b'?',b'?',b'?',b'?',b'?',b'?',4.6,56.0,16.0,4.6,NaN,b'no',b'LIVE'
4,31.0,b'female',b'yes',b'no',b'no',b'no',b'no',b'yes',b'no',b'no',b'no',b'no',b'no',1.0,85.0,20.0,4.0,100.0,b'no',b'LIVE'


In [4]:
for df in train_dfs:
    preprocess_hepatitis_datasets(df)

In [12]:
# Check for missing values in the first training dataset
print("Missing values in train_dfs[0]:")

#print(train_dfs[0].isnull().sum())

train_dfs[0] = train_dfs[0].fillna(train_dfs[0].mean())

#print(train_dfs[0].isnull().sum())


Missing values in train_dfs[0]:


TypeError: Could not convert [b'femalefemalemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalemalefemalefemalefemalemalefemalefemalemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalefemalemalefemalefemalefemalemalefemalefemalemalefemalefemalefemalefemalefemalemalefemalefemalefemalefemalefemalefemalemalefemalefemalefemalefemalefemalefemalefemalefemalemalefemalefemalefemalefemalefemalefemalefemalefemalemalefemalefemalefemalefemalemalefemalefemalefemalefemalefemalefemalefemalefemale'
 b'nonoyesyesyesyesnonoyesnoyesyesnoyesnononoyesyesnoyesnonoyesnoyesyesnoyesyesyesnoyesyesyesnoyesyesyesnonononoyesyesyesyesnononononononononononoye